In [1]:
import awkward as ak
import pandas as pd
import numpy as np
import fastjet
import uproot
import vector
vector.register_awkward()

Read in stuff

In [2]:
f = uproot.open("filelist/badFromLuca/miniaod_1032.root")
events = f['Events']
arrays = events.arrays()

In [3]:
Cands = ak.zip({
    "pt": arrays['PFCands_trkPt'],
    "eta": arrays['PFCands_trkEta'],
    "phi": arrays['PFCands_trkPhi'],
    "mass": arrays['PFCands_mass']
}, with_name="Momentum4D")
cut = (arrays['PFCands_fromPV'] > 1) & \
         (arrays['PFCands_trkPt'] >= 0.7) & \
         (abs(arrays['PFCands_trkEta']) <= 2.5) & \
         (abs(arrays['PFCands_dz']) < 10) & \
         (arrays['PFCands_dzErr'] < 0.05)
Cleaned_cands = Cands[cut]
Cleaned_cands = ak.packed(Cleaned_cands)

tracks = Cleaned_cands

In [4]:
Jets = ak.zip({
        "pt": arrays['Jet_pt'],
        "eta": arrays['Jet_eta'],
        "phi": arrays['Jet_phi'],
        "mass": arrays['Jet_mass'],
        "jetId": arrays['Jet_jetId']
    })
jetCut = (Jets.pt > 30) & (abs(Jets.eta)<2.4)
ak4jets = Jets[jetCut]
ht = ak.sum(ak4jets.pt,axis=-1)
trigger = arrays['HLT_PFHT1050']
tracks = tracks[trigger]

Define FastJets with anti-kt 1.5

In [5]:
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.5)        

In [19]:
print(tracks.pt)

[[0.757, 0.741, 0.786, 0.881, 0.828, 0.895, ... 3.46, 2.86, 0.922, 1.48, 1.84, 1.86]]


This will fail<br>
min_pt = 0 and = 150 both fail

In [13]:
cluster = fastjet.ClusterSequence(tracks, jetdef)
ak_inclusive_jets = ak.with_name(cluster.inclusive_jets(min_pt=150),"Momentum4D") 
#ak_inclusive_cluster = ak.with_name(cluster.constituents(),"Momentum4D")

In [17]:
tracks[0].pt.type

103 * float32

In [16]:
ak_inclusive_jets[0].pt

<Array [292, 504, 511] type='3 * float64'>

This works<br>

In [9]:
test = []
for event in tracks:
    test.append([track for track in event])
a = ak.Array(test, with_name='Momentum4D')
cluster = fastjet.ClusterSequence(a, jetdef)
ak_inclusive_jets = ak.with_name(cluster.inclusive_jets(min_pt=150),"Momentum4D") 
ak_inclusive_cluster = ak.with_name(cluster.constituents(min_pt=150),"Momentum4D")

All these types seem the same, so what's going on?

In [36]:
print(tracks[0])
print(type(tracks[0]))
print(type(tracks))

[{pt: 0.757, eta: 2, phi: 0.438, mass: 0.14}, ... phi: -0.557, mass: 0.14}]
<class 'vector._backends.awkward_.MomentumArray4D'>
<class 'vector._backends.awkward_.MomentumArray4D'>


In [37]:
print(a[0])
print(type(a[0]))
print(type(a))

[{pt: 0.757, eta: 2, phi: 0.438, mass: 0.14}, ... phi: -0.557, mass: 0.14}]
<class 'vector._backends.awkward_.MomentumArray4D'>
<class 'vector._backends.awkward_.MomentumArray4D'>


The elements are identical

In [42]:
ak.all(a == tracks)

True